In [294]:
import pandas as pd
from sqlalchemy import create_engine

In [295]:
# TODO : Extract data from Postgres and then disconnect connection

# Define your PostgreSQL connection parameters
db_params = {
    'dbname': 'mydb',
    'user': 'postgres',
    'password': '1',
    'host': 'localhost',
    'port': '5432'
}

# Create the connection URL
connection_url = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Create the connection engine
engine = create_engine(connection_url)

# Specify your schema
schema_name = 'test_task'


# TODO : accounts
sql_query_accounts = f"SELECT * FROM \"{schema_name}\".accounts"
accounts = pd.read_sql(sql_query_accounts, engine)
accounts = accounts.rename(columns={"id":"profile_id"})


# TODO : posts
sql_query_posts = f"SELECT * FROM \"{schema_name}\".posts"
posts = pd.read_sql(sql_query_posts, engine)
posts = posts.rename(columns={"id":"post_id"})


# TODO : sources_for_followers
sql_query_sources_for_followers = f"SELECT * FROM \"{schema_name}\".sources_for_followers"
sources_for_followers = pd.read_sql(sql_query_sources_for_followers, engine)


# Close the connection
engine.dispose()

In [296]:
# TODO : Need to change  posts.<profile_id>  to accounts.<alt_id> because values not matched

# Merge posts with accounts on the given condition
merged_df = posts.merge(accounts, left_on='profile_id', right_on='id_alt', how='left')

# Replace the 'profile_id' in posts dataframe
posts['profile_id'] = merged_df['profile_id_x'].where(merged_df['profile_id_y'].isna(), merged_df['profile_id_y'])

# First Tables analyse 

##  ---> accounts + sources_for_followers

In [297]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

In [298]:
# TODO : accounts full info (accounts + source for followers)

account_full_info = accounts.merge(sources_for_followers, how='outer', on='_id')


test chart 1. Username and all followers, with legend (True or false in verified)

In [299]:
# TODO : bar plotly diagram to see accounts and their followers #media_and_followers(account_full_info)

def media_and_followers(account_full_info):
    account_full_info = account_full_info.sort_values(by='followers_count', ascending=False)

    # Define colors based on 'is_verified'
    account_full_info['color'] = account_full_info['is_verified'].map({
        True: 'green',
        False: 'red'
    }).fillna('orange')


    fig = px.bar(
        account_full_info, 
        x='username', 
        y='followers_count', 
        title="username vs. followers (divided on verified accounts)",
        color_discrete_map={
            True: 'green',
            False: 'red',
            None: 'orange'
        },
        color='is_verified'
    )

    # Show plot
    fig.show()

    text0 = f"**->Here we can see the highest media channels by their reach of subscribers.**"
    display(Markdown(text0))

## ---> posts stats by comments and posts (merge with accounts info)

## 1. > all activity over time on line chart 

In [300]:
# TODO : (All posts activity by time range)  #posts_and_comments_OverTime(posts)

def posts_and_comments_OverTime(posts):
    date_period = posts.copy(deep=True)
    date_period['created_time'] = pd.to_datetime(date_period['created_time'], utc=True)#.dt.tz_convert('UTC').dt.strftime("%Y-%m-%d")#.apply(pd.to_datetime)
    date_period = date_period.set_index('created_time')

    grouped = date_period.groupby([date_period.index.year.rename('Year'), 
                                date_period.index.month.rename('Month'), 
                                date_period.index.day.rename('Day')])\
                        .agg(all_posts=pd.NamedAgg(column='post_id', aggfunc='count'),
                            all_comments=pd.NamedAgg(column='comments_count', aggfunc='sum')).reset_index()

    grouped['Date'] = pd.to_datetime(grouped[['Year', 'Month', 'Day']])


    # TODO : double subplot for STATS OVER TIME

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=grouped['Date'], y=grouped['all_posts'], name="All Posts"), secondary_y=False)
    fig.add_trace(go.Scatter(x=grouped['Date'], y=grouped['all_comments'], name="All Comments"), secondary_y=True)

    fig.update_layout(title_text="Posts and Comments over Time")
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="All Posts", secondary_y=False)
    fig.update_yaxes(title_text="All Comments", secondary_y=True)

    fig.show()

    text1 = f"**->On this chart we can see activity of posts number and how many comments activity they scored.**\n\n**->You can roughly see that every 5-10 days there was a post that became a mini-trend or just went viral.**"
    display(Markdown(text1))

## 2. > media sources activity on  chart

In [301]:
# TODO : we have NaN values in username -- for every NaN changing it for a new unique value   ## name_generator() // fill_with_unique_name(val)

# Create a generator that yields the new unique names
def name_generator():
    count = 1
    while True:
        yield f'without name {count}'
        count += 1

gen = name_generator()

# Create a custom function that fills NaN values using the generator
def fill_with_unique_name(val):
    if pd.isna(val):
        return next(gen)
    else:
        return val



In [302]:
# TODO : merge and transform data for media sources activity chart

posts_info_withAks = posts.merge(account_full_info, how='outer', on='profile_id')

Aks_groupbyPosts = posts_info_withAks.groupby(['profile_id', 'username', 'is_verified', '_id', 'followers_count'], dropna=False)\
    [['post_id','comments_count']].agg(
        all_posts = ('post_id','count'),
        all_comments = ('comments_count','sum')).reset_index()


# Apply this function to the column
Aks_groupbyPosts['username'] = Aks_groupbyPosts['username'].apply(fill_with_unique_name)


# TODO : 
Aks_groupbyPosts[['followers_count',	'all_posts',	'all_comments']] = Aks_groupbyPosts[['followers_count',	'all_posts',	'all_comments']].astype('int')

Aks_groupbyPosts['%comments coverage for 5 months'] = (Aks_groupbyPosts['all_comments'] /Aks_groupbyPosts['followers_count']) *100

2.1 -> All Media Activity 5 months total

In [303]:
# TODO :First Chart: "All media activity"  #Media_activity_total_5Months(Aks_groupbyPosts)

def Media_activity_total_5Months(Aks_groupbyPosts):
    Aks_groupbyPosts = Aks_groupbyPosts.sort_values(by='followers_count', ascending=False)

    # Define colors based on 'is_verified'
    Aks_groupbyPosts['color'] = Aks_groupbyPosts['is_verified'].map({
        True: 'green',
        False: 'red'}).fillna('orange')

    # Create the plot
    fig1 = go.Figure()

    # Add the actual data traces with the desired names for the legend
    fig1.add_trace(go.Bar(x=Aks_groupbyPosts['username'], y=Aks_groupbyPosts['all_posts'], 
                        name='All Posts',
                        marker_color=Aks_groupbyPosts['color']))

    fig1.add_trace(go.Scatter(x=Aks_groupbyPosts['username'], y=Aks_groupbyPosts['all_comments'], 
                            yaxis='y2', name='All Comments',
                            line=dict(color='blue')))

    fig1.update_layout(title='All Media Activity 5 months total', 
                    xaxis_title='Username', 
                    yaxis_title='All Posts',
                    yaxis2=dict(title='All Comments', overlaying='y', side='right'))

    fig1.show()

    text2 = f"**->On this chart we can see: How many posts and comments has every media sources (5 months)**\n\n**Red - not verified**\n\n**Green - verified**\n\n**Orange - don't know**"
    display(Markdown(text2))

2.2 -> %Comments Coverage by source , 5 Months

In [304]:
# TODO : second chart Audience Activity # audience_activity(Aks_groupbyPosts)

def audience_activity(Aks_groupbyPosts):
    color_map = {
        True: 'green',
        False: 'red',
        np.nan: 'orange'
    }

    # Create a color column
    Aks_groupbyPosts['color'] = Aks_groupbyPosts['is_verified'].map(color_map)

    fig2 = go.Figure()

    # Bar trace with the color mapping
    fig2.add_trace(go.Bar(x=Aks_groupbyPosts['username'], 
                        y=Aks_groupbyPosts['followers_count'], 
                        name='Followers Count', 
                        marker=dict(color=Aks_groupbyPosts['color'])))

    # Line trace
    fig2.add_trace(go.Scatter(x=Aks_groupbyPosts['username'], 
                            y=Aks_groupbyPosts['%comments coverage for 5 months'], 
                            name='%Comments Coverage for 5 Months', 
                            yaxis='y2', 
                            line=dict(color='blue')))

    # Layout update
    fig2.update_layout(title='Audience Activity', 
                    xaxis_title='Username', 
                    yaxis_title='Followers Count',
                    yaxis2=dict(title='%Comments Coverage for 5 Months', overlaying='y', side='right'))

    # Show the figure
    fig2.show()

    text3 = f"**->On this chart we can see: How many posts and which subscribers activity has every media sources (5 months)**\n\n**Red - not verified**\n\n**Green - verified**\n\n**Orange - don't know**"
    display(Markdown(text3))

## 3. -> All media activity on period charts 

In [305]:
# TODO : df operations
posts_info_withAks['created_time'] = pd.to_datetime(posts_info_withAks['created_time'], utc=True).dt.tz_convert('UTC').dt.strftime("%Y-%m-%d")#.apply(pd.to_datetime)

Aks_groupbyPosts_periods = posts_info_withAks.groupby(['profile_id', 'username', 'is_verified', '_id', 'followers_count','created_time'], dropna=False)\
    [['post_id','comments_count']].agg(
        all_posts = ('post_id','count'),
        all_comments = ('comments_count','sum')).reset_index()


condition = Aks_groupbyPosts_periods['username'].isna()
Aks_groupbyPosts_periods['username'] = np.where(
    condition,
    Aks_groupbyPosts_periods['profile_id'].astype(str).str[-4:],
    Aks_groupbyPosts_periods['username'])

3. -> All posts by every account on chart

In [306]:
# TODO : third CHART sum posts on period line divided by media sources. # SumPosts_OnPeriod_Divided_byMedia(Aks_groupbyPosts_periods)

def SumPosts_OnPeriod_Divided_byMedia(Aks_groupbyPosts_periods):
    fig = go.Figure()

    color_map = {
        True: 'green',
        False: 'red'
    }

    for user in Aks_groupbyPosts_periods['username'].unique():
        user_Aks_groupbyPosts_periods = Aks_groupbyPosts_periods[Aks_groupbyPosts_periods['username'] == user]
        # Use the first row's 'is_verified' status to determine the color for this user
        is_verified_val = user_Aks_groupbyPosts_periods['is_verified'].iloc[0]
        if pd.isna(is_verified_val):
            color = 'orange'
        else:
            color = color_map[is_verified_val]
        
        fig.add_trace(go.Scatter(
            x=user_Aks_groupbyPosts_periods['created_time'], 
            y=user_Aks_groupbyPosts_periods['all_posts'], 
            name=user,
            mode='lines',
            line=dict(color=color),
            hovertemplate=(
                f"Username: {user}<br>"
                f"Sum Posts: {{y}}<br>"
                f"Date: {{x}}"
            )
        ))

    # Add a legend to describe the colors
    annotations = [
        dict(x=1.05, y=1, xref='paper', yref='paper', showarrow=False, text="Colors:", font=dict(size=15)),
        dict(x=1.05, y=0.9, xref='paper', yref='paper', showarrow=False, text="Verified: Green"),
        dict(x=1.05, y=0.8, xref='paper', yref='paper', showarrow=False, text="Not Verified: Red"),
        dict(x=1.05, y=0.7, xref='paper', yref='paper', showarrow=False, text="Unknown: Orange"),
    ]

    fig.update_layout(annotations=annotations)
    fig.update_layout(
        title="sum posts on period line divided by media sources",
        annotations=annotations
    )
    fig.show()

    text4 = f"**On chart we can see sum posts on period line divided by media sources.**\n\n**Red - not verified**\n\n**Green - verified**\n\n**Orange - don't know**\n\n**But on this chart hard to see something**"
    display(Markdown(text4))

3.2 -> all comments by every username on period

In [307]:
# TODO : fourth CHART sum comments on period line divided by media sources

def SumComments_OnPeriod_Divided_byMedia(Aks_groupbyPosts_periods):
    fig = go.Figure()

    color_map = {
        True: 'green',
        False: 'red'
    }

    for user in Aks_groupbyPosts_periods['username'].unique():
        user_Aks_groupbyPosts_periods = Aks_groupbyPosts_periods[Aks_groupbyPosts_periods['username'] == user]
        # Use the first row's 'is_verified' status to determine the color for this user
        is_verified_val = user_Aks_groupbyPosts_periods['is_verified'].iloc[0]
        if pd.isna(is_verified_val):
            color = 'orange'
        else:
            color = color_map[is_verified_val]
        
        fig.add_trace(go.Scatter(
            x=user_Aks_groupbyPosts_periods['created_time'], 
            y=user_Aks_groupbyPosts_periods['all_comments'], 
            name=user,
            mode='lines',
            line=dict(color=color),
            hovertemplate=(
                f"Username: {user}<br>"
                f"Sum Posts: {{y}}"
                f"Date: {{x}}"
            )
        ))

    # Add a legend to describe the colors
    annotations = [
        dict(x=1.05, y=1, xref='paper', yref='paper', showarrow=False, text="Colors:", font=dict(size=15)),
        dict(x=1.05, y=0.9, xref='paper', yref='paper', showarrow=False, text="Verified: Green"),
        dict(x=1.05, y=0.8, xref='paper', yref='paper', showarrow=False, text="Not Verified: Red"),
        dict(x=1.05, y=0.7, xref='paper', yref='paper', showarrow=False, text="Unknown: Orange"),
    ]

    fig.update_layout(annotations=annotations)
    fig.update_layout(
        title="sum comments on period line divided by media sources",
        annotations=annotations
    )
    fig.show()

    text4 = f"**On chart we can see sum comments on period line divided by media sources.**\n\n**Red - not verified**\n\n**Green - verified**\n\n**Orange - don't know**\n\n**But on this chart hard to see something**"
    display(Markdown(text4))

In [318]:
# TODO : Display Dataframe with Trends Posts # display_trends_posts()

def display_trends_posts():
    # Calculate the threshold using quantile
    comment_threshold = posts_info_withAks['comments_count'].quantile(0.95)

    # Filter the dataframe for posts that exceed this threshold
    peak_posts = posts_info_withAks[posts_info_withAks['comments_count'] > comment_threshold]

    # Display specific columns of interest for these high-comment posts
    return peak_posts[['created_time', 'username', 'description', 'text_original', 'comments_count']]

In [309]:
# TODO : Divide Media followers on quantiles and then grouping relevant info... #stats_byGrouped_Media(Aks_groupbyPosts_periods)

def stats_byGrouped_Media(Aks_groupbyPosts_periods):
    labels = ["Small media", "Medium media", "Large media", "Giant media"]
    Aks_groupbyPosts_periods['quantile_group'] = pd.qcut(Aks_groupbyPosts_periods['followers_count'], q=4, labels=labels).astype('str')
    group_size_quantiles = Aks_groupbyPosts_periods.copy(deep=True)

    group_size_quantiles['is_verified'] = np.where(group_size_quantiles['is_verified']==True, 1, 0)


    group_size_quantiles_groupby_Media = group_size_quantiles.groupby(['username','quantile_group','is_verified', 'followers_count'])[[ 'all_posts',	'all_comments', 'created_time']]\
        .agg(all_posts = ('all_posts', 'sum'), all_comments = ('all_comments', 'sum'), active_days = ('created_time', 'count')).reset_index()

    media_group_quantiles = group_size_quantiles_groupby_Media.groupby(['quantile_group'])[['is_verified', 'followers_count','all_posts', 'all_comments', 'active_days']].agg(
        all_accounts_inGroup = ('is_verified', 'count'),
        verified_accounts = ('is_verified', 'sum'),
        all_followers = ('followers_count', 'sum'),
        all_posts = ('all_posts', 'sum'),
        all_comments = ('all_comments', 'sum'),
        average_activeDays = ('active_days', 'mean')
    ).reset_index()


    media_group_quantiles['Total_comments_percentage'] = ((media_group_quantiles['all_comments'] / media_group_quantiles['all_comments'].sum())*100).round().astype(str) + '%'
    media_group_quantiles['Total_posts_percentage'] = ((media_group_quantiles['all_posts'] / media_group_quantiles['all_posts'].sum())*100).round().astype(str) + '%'
    media_group_quantiles = media_group_quantiles[['quantile_group', 'all_accounts_inGroup', 'verified_accounts',
        'all_followers', 'all_posts','Total_posts_percentage', 'all_comments', 'Total_comments_percentage','average_activeDays']]

    def highlight_rows(row):
        if row.name in [1, 2]:
            return ['background-color: green'] * len(row)
        return [''] * len(row)

    styled_df = media_group_quantiles.style.apply(highlight_rows, axis=1)
    display(styled_df)

#             !! Output result with key metrics !!

## -> 1. Chart "All MediaSources & followers" + legend for verified aks

In [310]:
media_and_followers(account_full_info)

**->Here we can see the highest media channels by their reach of subscribers.**

## -> 2. Chart "Posts and Comments" on timeline

In [311]:
posts_and_comments_OverTime(posts)

**->On this chart we can see activity of posts number and how many comments activity they scored.**

**->You can roughly see that every 5-10 days there was a post that became a mini-trend or just went viral.**

## -> 3. Chart "Media activity" total 5 month

In [312]:
Media_activity_total_5Months(Aks_groupbyPosts)

**->On this chart we can see: How many posts and comments has every media sources (5 months)**

**Red - not verified**

**Green - verified**

**Orange - don't know**

## -> 4. Chart "Followers" activity by every MediaSource

In [313]:
audience_activity(Aks_groupbyPosts)

**->On this chart we can see: How many posts and which subscribers activity has every media sources (5 months)**

**Red - not verified**

**Green - verified**

**Orange - don't know**

## -> 5. Chart "Sum Posts" by every media on timeline

In [314]:
SumPosts_OnPeriod_Divided_byMedia(Aks_groupbyPosts_periods)

**On chart we can see sum posts on period line divided by media sources.**

**Red - not verified**

**Green - verified**

**Orange - don't know**

**But on this chart hard to see something**

## -> 6. Chart "Sum comments" by every media on timeline

In [315]:
SumComments_OnPeriod_Divided_byMedia(Aks_groupbyPosts_periods)

**On chart we can see sum comments on period line divided by media sources.**

**Red - not verified**

**Green - verified**

**Orange - don't know**

**But on this chart hard to see something**

## -> 7. TRENDS! ... Top posts by comments. Dataframe output

In [320]:
display_trends_posts().head(5)

,created_time,username,description,text_original,comments_count
11,2023-04-24,russembangola,A página oficial da Embaixada da Rússia em Ang...,"Când o mamă se roagă pentru copiii ei, Dumneze...",98
16,2023-04-24,russembangola,A página oficial da Embaixada da Rússia em Ang...,# 🙏❤️[Hristos a purtat crucea ca orice suflet ...,73
27,2023-04-23,russembangola,A página oficial da Embaixada da Rússia em Ang...,"🌺Mai frumoasă rugăciune,Fie ca família ta să f...",88
43,2023-03-01,Prüfe-alles-glaube-wenig-denke-selbst,🙂 https://t.me/pruefealles & 🎬 https://rumble....,بثينة شعبان تتحدث لـRT حول تغير العلاقة بين سو...,118
52,2023-02-06,Prüfe-alles-glaube-wenig-denke-selbst,🙂 https://t.me/pruefealles & 🎬 https://rumble....,مباشر من سوريا.. لحظات عصيبة وحاسمة في البحث ع...,96


## -> 8. Divide media on quantiles and then grouping them with relevant information. Styled Dataframe

In [317]:
stats_byGrouped_Media(Aks_groupbyPosts_periods)

,quantile_group,all_accounts_inGroup,verified_accounts,all_followers,all_posts,Total_posts_percentage,all_comments,Total_comments_percentage,average_activeDays
0,Giant media,7,6,4983579,241,21.0%,1702,13.0%,8.285714
1,Large media,10,1,1024198,390,34.0%,4504,35.0%,9.200000
2,Medium media,6,1,318316,234,21.0%,4971,39.0%,13.500000
3,Small media,7,2,211992,273,24.0%,1716,13.0%,13.428571
